faiss事前準備:

miniconda_python(3.10版本):

https://docs.conda.io/projects/miniconda/en/latest/miniconda-other-installer-links.html

裝完打開anaconda prompt:

做一個環境```conda create -n VScode-env-3.10.13 python=3.10.13```

做完啟動```conda activate VScode-env-3.10.13```

安裝faiss插件

```conda install -c pytorch faiss-cpu=1.7.4 mkl=2021 blas=1.0=mkl```

```conda install -c pytorch/label/nightly faiss-cpu```

關掉prompt 重新開啟VScode 右上角kernel 選擇another kernel->python env->VScode-env-3.10.13

VScode商城:

python

jupyter

import和拿key

沒裝openai_SDK使用:```pip install openai```

沒裝tabulate使用:```pip install tabulate```

沒裝pandas:```pip install pandas```

沒設定key使用:```setx OPENAI_API_KEY “<yourkey>”```

```echo %OPENAI_API_KEY%```可以看你的KEY

In [22]:
import os
from openai import OpenAI
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')

# input在此輸入
# channel在此輸入(1.App 2.Email 3.comment 4.funtion calling)
# prompt type(1.origin 2.funtion calling)

In [23]:
input = "Can I change login password?"
channel = 4
prompt_type = 2

# instruction在此輸入

In [24]:
instruction_App = "You are InApp chatting bot from Flora customer service. You must answer questions related to Flora. We will give you one to three history questions and answers, You must answer based on the history. If there is 0 history question and answer, you must reply only a special mark:'*'"

instruction_Email = "You are email reply bot from Flora customer service. You must answer questions related to Flora. We will give you one to three history questions and answers, You must answer based on the history. If there is 0 history question and answer, you must reply only a special mark:'*'"

instruction_Comment = "You are Apple Appstore comment reply bot from Flora customer service. You must answer questions related to Flora. We will give you one to three history questions and answers, You must answer based on the history. If there is 0 history question and answer, you must reply only a special mark:'*'"

instruction_Funtion = "You are Flora customer service. You must answer based on the history questions and answers from Flora's database. If there is 0 history question and answer, you must reply only a special mark:'*'"

In [25]:
instruction = ""
if channel == 1 :
    instruction = instruction_App
elif channel == 2 :
    instruction = instruction_Email
elif channel == 3 :
    instruction = instruction_Comment
elif channel == 4 :
    instruction = instruction_Funtion

In [ ]:
instruction += "\nYou must follow the SOP: \n"

# The rule that Assistant should follow
SOP = [
    "1. Based on history Q&A to answer.",
    "2. If the customer's problem still can not be solved after you replied, reply only a special mark:'*'.",
]

for i in SOP:
    instruction = instruction + i + "\n"

print(instruction)

先準備faiss

讀embeddings資料

並把dataframe轉為numpy

In [26]:
import pandas as pd
import numpy as np

csv_file = 'data\embeddingsV3_3072.csv'
df = pd.read_csv(csv_file)
data = df.to_numpy()
df.head(5)

確認內容

In [27]:
print(data.shape[0])
print(type(data))
print(data.dtype)
print(data[0][0])

寫入維度 並把data加入faiss的index

In [28]:
import faiss

index = faiss.IndexFlatL2(data.shape[1])
index.add(data)

做問題的embedding

TOP-k的設定並查找問題

拿答案

In [29]:
def get_similar(input):
    response = client.embeddings.create(input=input, model="text-embedding-3-large", dimensions=3072)
    question = np.array(response.data[0].embedding, ndmin=2)
    k = 3
    distances, indices = index.search(question, k)
    csv_question = 'data\manual.csv'
    qdf = pd.read_csv(csv_question)
    qarray = qdf.to_numpy()
    answer = [(qarray[i], float(dist)) for dist, i in zip(distances[0], indices[0])]
    history = ""
    min_distance = 1
    Qcount = 0
    for i in range(0,k):
        if answer[i][1]<min_distance:
            Qcount+=1
            history = history +str(i+1)+". "+ answer[i][0][0]+"\n"+answer[i][0][1]+"\n"
    return Qcount, history

距離設定dist內才顯示

# Prompt內容在此

In [ ]:
similar = get_similar(input)
if prompt_type == 1:
    prompt = "Here is "+str(similar[0])+" history questions and answers related to this question according to Flora's database\n"+similar[1]+"Here is my question:\n"+input
elif prompt_type == 2:
    prompt = input
print(prompt)

創assistant,thread,message!

In [31]:
from tabulate import tabulate

assistant = client.beta.assistants.create(
    name="Flora's Customer Service",
    instructions=instruction,
    model="gpt-3.5-turbo-1106",
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_history_question_and_answer",
                "description": "Get 1 to 3 history questions and answers that is similar to current question",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "current_question": {
                            "type": "string",
                            "description": "user's question",
                        },
                    },
                    "required": ["current_question"],
                },
            },
        }
    ]
)

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content= prompt
)


沒裝tabulate很醜

tabulate文件:https://pypi.org/project/tabulate/

表示assistant,thread,message內容

In [ ]:
print(tabulate(assistant,headers=['assistant','content'],tablefmt="simple_outline"))

In [ ]:
print(tabulate(thread,headers=['thread','content'],tablefmt="simple_outline"))

In [ ]:
print(tabulate(message,headers=['message','content'],tablefmt="simple_outline"))

這次的run

可以使用新instruction複寫assisstant裡的instruction!

like:

```
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)
```


In [35]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)


查看run的狀態!

以上如果還沒跑完會顯示queued!

以上如果跑完會顯示completed!

In [ ]:
# run = client.beta.threads.runs.retrieve(
#   thread_id=thread.id,
#   run_id=run.id
# )
# print(tabulate(run,headers=['run','content'],tablefmt='simple_outline'))

while run.status == "queued" or run.status == "in_progress":
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
print(tabulate(run,headers=['run','content'],tablefmt='simple_outline'))

In [ ]:
# if function calling is required, print user input
try:
    print(run.required_action.submit_tool_outputs.tool_calls[0].function.arguments[21:-2])

except AttributeError:
    print("No function calling")


In [40]:
try:
  if run.required_action.submit_tool_outputs.tool_calls[0].function.name == "get_history_question_and_answer":
    similar = get_similar(run.required_action.submit_tool_outputs.tool_calls[0].function.arguments[21:-2])
    run = client.beta.threads.runs.submit_tool_outputs(
      thread_id=thread.id,
      run_id=run.id,
      tool_outputs=[
        {
          "tool_call_id": run.required_action.submit_tool_outputs.tool_calls[0].id,
          "output": similar[1]
        }
      ]
    )
except AttributeError:
  print("No function calling")

跑完後打開查看history messages!

In [ ]:
import time

time.sleep(5)
messages = client.beta.threads.messages.list(
  thread_id=thread.id,
  order='asc'
)
for i in messages.data:
    print(tabulate(i,tablefmt='simple_outline'))

訊息顯示

In [ ]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id,
  order='asc'
)
for i in messages.data:
  print(i.content[0].text.value)
  print("\n\n\n")

# Delete Assistant

In [ ]:
response = client.beta.assistants.delete(assistant.id)
time.sleep(5)
print(response)